In [1]:
import typing
import httpx
from pydantic import BaseModel


class GithubIssue(BaseModel):
    title: str
    issue_url: str


USERNAME = "michalnik"
GITHUB_API = "https://api.github.com"


def get_github_issues(api: str, username: str) -> typing.Iterable[GithubIssue]:
    search_url = f"{api}/search/issues"
    query_params = {"q": f"commenter:{username}", "sort": "created", "order": "desc"}
    response = httpx.get(search_url, params=query_params, follow_redirects=True)
    try:
        for item in response.json()["items"]:
            yield GithubIssue(title=item["title"], issue_url= item["html_url"])
    except Exception as exc:
        print(f"Something bad happened while reading data from github: {type(exc)} => {exc.args}")
        raise SystemError("Going down...")

In [2]:
import os
from functools import partial
from asgiref.sync import sync_to_async

import django
from django.utils import timezone


# set paths
#     1) to locate project root to enable founding of django project and its applications
#     2) to locate django project settings for django setup
os.sys.path.insert(0, os.getenv("PROJECT_ROOT"))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "www.settings")

# startup of django
django.setup()


# import Activity model
from activity.models import Activity


# set initial data from github
for issue in get_github_issues(GITHUB_API, USERNAME):
    # we are in async context(natural for notebook) so we need to call sync_to_async for Django ORM
    #    - because of sync_to_async signature it is required to create new sync function to call it later without any arguments
    #    - functools.partial can do the work ...
    create_activity = partial(Activity.objects.create, created_at=timezone.now(), title=issue.title, issue_url=issue.issue_url, comment_url="")
    await sync_to_async(create_activity, thread_sensitive=True)()
